In [7]:
import torch
from torch import nn
import torchvision
import torchvision.models as models



In [2]:
#vgg16_model=models.vgg16(pretrained=True)
vgg16_model = models.vgg16(weights='VGG16_Weights.DEFAULT')
modules=list(vgg16_model.children())[:-2]
modules = modules + [torch.nn.Conv2d(in_channels=512,out_channels=64,kernel_size=3,stride=1,padding=1),torch.nn.ReLU(inplace=True),torch.nn.AdaptiveAvgPool2d(output_size=(128, 128))]
vgg16_model=nn.Sequential(*modules)
print(vgg16_model)

img = torch.randn((100,3,256,256))
features_var=vgg16_model(img)
features=features_var.data
features=features.data
print(features.shape)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
#print(list(models.vgg16(weights='VGG16_Weights.DEFAULT').children())[-3])
print(list(models.vgg16(weights='VGG16_Weights.DEFAULT').children())[-2])
print(list(models.vgg16(weights='VGG16_Weights.DEFAULT').children())[-1])

AdaptiveAvgPool2d(output_size=(7, 7))
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [4]:
vgg16_model = models.vgg16(weights='VGG16_Weights.DEFAULT')
modules=list(vgg16_model.children())[:-2]
modules = modules + [torch.nn.Conv2d(in_channels=512,out_channels=64,kernel_size=3,stride=1,padding=1),
                     torch.nn.ReLU(inplace=True),
                     torch.nn.AdaptiveAvgPool2d(output_size=(128, 128)),
                    ]

vgg16_model=torch.nn.Sequential(*modules)

img = torch.randn((100,3,256,256))
features = vgg16_model(img)

print(features.shape)

torch.Size([100, 64, 128, 128])


In [9]:
class SharedConvolutionalLayers(nn.Module):
    def __init__(self, out_channels):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3,48,kernel_size=3,stride=1,padding=1)
        self.conv2 = torch.nn.Conv2d(48,128,kernel_size=3,stride=2,padding=1)
        self.conv3 = torch.nn.Conv2d(128,out_channels,kernel_size=3,stride=2,padding=1)

    def forward(self,x):
        shared_output = []
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.relu(self.conv3(x))

        shared_output.append(x)

        return shared_output

img = torch.randn((100,3,256,256))
mmodel = SharedConvolutionalLayers(128)
features = mmodel(img)

print(len(features))
print(features[0].shape)

1
torch.Size([100, 128, 64, 64])


In [14]:
class SharedConvLayersVGG(nn.Module):
    def __init__(self,out_channels,out_size=64):
        super().__init__()
        vgg16_back = torchvision.models.vgg16(weights="VGG16_Weights.DEFAULT")
        modules = list(vgg16_back.children())[:-2]
        module_list = modules + [torch.nn.Conv2d(in_channels=512,out_channels=out_channels,kernel_size=3,stride=1,padding=1),
                                 torch.nn.ReLU(inplace=True),
                                 torch.nn.AdaptiveAvgPool2d(output_size=(out_size, out_size))]
        self.share_conv_layers = nn.Sequential(*module_list)

    def forward(self,x):
        shared_output = []
        out = self.share_conv_layers(x)
        shared_output.append(out)
        return shared_output

img = torch.randn((100,3,256,256))
model2 = SharedConvLayersVGG(128)
features2 = model2(img)

print(len(features2))
print(features2[0].shape)

1
torch.Size([100, 128, 64, 64])


In [18]:
class SharedConvLayersVGG(nn.Module):
    def __init__(self,out_channels,out_size=64):
        super().__init__()
        vgg16 = torchvision.models.vgg16(weights="VGG16_Weights.DEFAULT").requires_grad_(False)
        modules = list(vgg16.children())[:-2]
        self.vgg16_back = nn.Sequential(*modules)

        module_list = [torch.nn.Conv2d(in_channels=512,out_channels=out_channels,kernel_size=3,stride=1,padding=1),
                       torch.nn.ReLU(inplace=True),
                       torch.nn.AdaptiveAvgPool2d(output_size=(out_size, out_size))]
        self.share_conv_layers = nn.Sequential(*module_list)

    def forward(self,x):
        shared_output = []
        h = self.vgg16_back(x)
        out = self.share_conv_layers(h)
        shared_output.append(out)
        return shared_output

img = torch.randn((100,3,256,256))
model2 = SharedConvLayersVGG(128)
features2 = model2(img)

print(len(features2))
print(features2[0].shape)

1
torch.Size([100, 128, 64, 64])


In [ ]:
img_tensor = torch.randn((8,3,128,128))
ll = []
for feature in img_tensor:
    h = torch.nn.Conv2d(3,24,kernel_size=3,padding=0,stride=3)(feature)
    ll.append(h)
print(len(ll),ll[0].shape)
alter = torch.stack(ll,dim=0)
print(len(alter))
print()

out = vgg16_model(img_tensor)
print(out.shape)
alter2 = [out]
print(len(alter2))
print(alter2[0].shape)